# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weatherdata = pd.read_csv("../Output_data/weather_data_final.csv")
weatherdata.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Aklavik,90,CA,1584160798,67,68.22,-135.01,23.00,2.24
1,Kavieng,8,PG,1584160401,76,-2.57,150.80,83.89,15.12
2,Punta Arenas,78,CL,1584160585,66,-53.15,-70.92,50.00,19.46
3,Padang,38,ID,1584160798,73,-0.95,100.35,82.18,1.97
4,Nanortalik,100,GL,1584160668,72,60.14,-45.24,23.16,15.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weatherdata[["Lat", "Lon"]]
humidity = weatherdata["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=weatherdata["Humidity"].max(),
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
weatherdata[["Max Temp","Wind Speed","Cloudiness"]].astype(float)
weather_condition = weatherdata.loc[(weatherdata["Max Temp"] < 80)] 
weather_condition = weather_condition.loc[(weatherdata["Max Temp"] > 70)]
weather_condition = weather_condition.loc[(weatherdata["Wind Speed"] < 10)]
weather_condition = weather_condition.loc[(weatherdata["Cloudiness"] == 0)]
hotel_df = weather_condition.dropna().set_index("City").reset_index()
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed
0,Arraial do Cabo,0,BR,1584160803,85,-22.97,-42.03,77.09,9.98
1,Barra de Tecoanapa,0,MX,1584160807,88,16.52,-98.75,72.64,2.84
2,Chui,0,UY,1584160816,88,-33.70,-53.46,70.23,9.08
3,Bogandé,0,BF,1584160820,12,12.97,-0.14,72.95,9.48
4,Kayes,0,ML,1584160828,9,14.00,-11.00,71.31,1.92
5,San Patricio,0,MX,1584160832,85,19.22,-104.70,75.02,5.44
6,Mattru,0,SL,1584160713,92,7.62,-11.83,76.01,2.55
7,Oussouye,0,SN,1584160843,44,12.48,-16.55,78.80,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""

In [7]:
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
for index,row in hotel_df.iterrows():
    # get lat, lon from df
    lon = row["Lon"]
    lat = row["Lat"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lon}"
    
    # Use the search term: "International Airport" and our lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
#     print the hotel_name url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    hotel_name = hotel_name.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        pass
        
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
0,Arraial do Cabo,0,BR,1584160803,85,-22.97,-42.03,77.09,9.98,Hotel da Canoa
1,Barra de Tecoanapa,0,MX,1584160807,88,16.52,-98.75,72.64,2.84,Hotel Garapacho
2,Chui,0,UY,1584160816,88,-33.70,-53.46,70.23,9.08,Turis Firper Hotel
3,Bogandé,0,BF,1584160820,12,12.97,-0.14,72.95,9.48,
4,Kayes,0,ML,1584160828,9,14.00,-11.00,71.31,1.92,
5,San Patricio,0,MX,1584160832,85,19.22,-104.70,75.02,5.44,Grand Isla Navidad Resort
6,Mattru,0,SL,1584160713,92,7.62,-11.83,76.01,2.55,
7,Oussouye,0,SN,1584160843,44,12.48,-16.55,78.80,9.17,Hotel Carabane


In [8]:
# Remove rows with empty cells
hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")].set_index("City").reset_index()
hotel_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lon,Max Temp,Wind Speed,Hotel Name
0,Arraial do Cabo,0,BR,1584160803,85,-22.97,-42.03,77.09,9.98,Hotel da Canoa
1,Barra de Tecoanapa,0,MX,1584160807,88,16.52,-98.75,72.64,2.84,Hotel Garapacho
2,Chui,0,UY,1584160816,88,-33.70,-53.46,70.23,9.08,Turis Firper Hotel
3,San Patricio,0,MX,1584160832,85,19.22,-104.70,75.02,5.44,Grand Isla Navidad Resort
4,Oussouye,0,SN,1584160843,44,12.48,-16.55,78.80,9.17,Hotel Carabane


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [10]:

fig = gmaps.figure(map_type = "TERRAIN")
markers = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig
# Display Map

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))